In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("CKD.csv")
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 
                     'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes','appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [9]:
#standardisation
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid={'n_neighbors':[3,5,7],'algorithm':['auto','ball_tree','kd_tree'],'metric':['minkowski'],'p':[2]}
grid= GridSearchCV(KNeighborsClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\keert\Anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                         'metric': ['minkowski'], 'n_neighbors': [3, 5, 7],
                         'p': [2]},
             scoring='f1_weighted', verbose=3)

In [17]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [19]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print ("The f1-macro value for best parameter {}:". format(grid.best_params_),f1_macro)
print ("The confusion matrix:\n",cm)
print ("The report:\n",clf_report)

The f1-macro value for best parameter {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 3, 'p': 2}: 0.9587114337568058
The confusion matrix:
 [[45  0]
 [ 5 70]]
The report:
               precision    recall  f1-score   support

       False       0.90      1.00      0.95        45
        True       1.00      0.93      0.97        75

    accuracy                           0.96       120
   macro avg       0.95      0.97      0.96       120
weighted avg       0.96      0.96      0.96       120



In [21]:
#roc-auc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013651,0.004046,0.303042,0.008404,auto,minkowski,3,2,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.02337,1
1,0.008085,0.003907,0.296361,0.004582,auto,minkowski,5,2,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.964572,0.929513,0.964572,0.963912,0.957428,0.01396,4
2,0.007762,0.002015,0.132959,0.139397,auto,minkowski,7,2,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.964572,0.947015,0.929513,0.964572,0.963912,0.953917,0.01393,7
3,0.012085,0.003776,0.032747,0.008423,ball_tree,minkowski,3,2,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.02337,1
4,0.007452,0.002298,0.025167,0.005392,ball_tree,minkowski,5,2,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.964572,0.964572,0.929513,0.964572,0.963912,0.957428,0.01396,4
5,0.006547,0.002409,0.030119,0.007452,ball_tree,minkowski,7,2,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.964572,0.947015,0.929513,0.964572,0.963912,0.953917,0.01393,7
6,0.005939,0.000798,0.028607,0.006487,kd_tree,minkowski,3,2,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.964572,0.982221,0.929513,0.964572,1.000000,0.968176,0.02337,1
7,0.006400,0.001004,0.027885,0.003570,kd_tree,minkowski,5,2,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.964572,0.964572,0.929513,0.964572,0.963912,0.957428,0.01396,4
8,0.007398,0.001714,0.019178,0.002385,kd_tree,minkowski,7,2,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.964572,0.947015,0.929513,0.964572,0.963912,0.953917,0.01393,7


In [23]:
grid.predict([[15,60,3,0,86,15,0.6,138,4,11,33,7700,3.8,0,0,0,0,1,1,0,0,1,1,0,1,0,0]])

array([ True])

In [25]:
h=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(h)

1.0


In [29]:
grid.predict([[15,80,0,0,93,17,0.9,136,3.9,16.7,50,6200,5.2,1,0,0,0,1,1,0,0,0,0,0,1,0,0]])

array([ True])